<a href="https://colab.research.google.com/github/Skander28/Models/blob/main/Data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
  !nvidia-smi
else:
  print('No GPU available, using the CPU instead.')
  device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4
Wed Mar 22 14:53:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    27W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                            

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/

 Classroom  'Colab Notebooks'   dialect_dataset.csv   messages.csv


In [ ]:
import pandas as pd
tweets_ids = pd.read_csv('/content/drive/MyDrive/dialect_dataset.csv')

In [ ]:
# Read the given dataset which contain the Ids and the labels
df_clean = pd.read_csv('/content/drive/MyDrive/messages.csv',
                 lineterminator='\n')

column_names = ['id', 'tweets']  # list of column names

df_clean.columns = column_names
df_clean.head()

,id,tweets
0,1.175358e+18,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .
1,1.175416e+18,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...
2,1.175450e+18,@KanaanRema مبين من كلامه خليجي
3,1.175471e+18,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐
4,1.175497e+18,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺


In [ ]:
tweets_dataset = pd.merge(tweets_ids, df_clean, on='id', how='inner')
tweets_dataset.head()

,id,dialect,tweets
0,1175358310087892992,IQ,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .
1,1175416117793349632,IQ,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...
2,1175450108898565888,IQ,@KanaanRema مبين من كلامه خليجي
3,1175471073770573824,IQ,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐
4,1175496913145217024,IQ,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺


# **Text Pre-processing**

In [ ]:
!pip install PyArabic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 KB 6.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
import string
import sys
import pyarabic.araby as araby
import nltk 
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
from textblob import Word
from sklearn import preprocessing

In [ ]:
filtered_df = tweets_dataset[tweets_dataset['dialect'].isin(['TN','DZ','MA','LY'])]
filtered_df.head()

,id,dialect,tweets
15497,1009754958479151232,LY,@toha_Altomy @gy_yah قليلين ادب ومنافقين. لو ا...
15498,1009794751548313600,LY,@AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ...
15499,1019989115490787200,LY,@smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بير...
15500,1035479791758135168,LY,@AboryPro @lyranoo85 رانيا عقليتك متخلفة. اولا...
15501,1035481122921164800,LY,@lyranoo85 شكلك متعقدة علشان الراجل لي تحبيه ا...


## **Text Cleaning**

### *Removing Punctuations and Symbols*

In [ ]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

In [ ]:
def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [ ]:
filtered_df['tweets'] = filtered_df['tweets'].apply(remove_punctuations)
filtered_df.head()

<ipython-input-18-d89cc7b25867>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['tweets'] = filtered_df['tweets'].apply(remove_punctuations)


,id,dialect,tweets
15497,1009754958479151232,LY,tohaAltomy gyyah قليلين ادب ومنافقين لو اختهم ...
15498,1009794751548313600,LY,AlmFaisal 😂😂 الليبيين متقلبين\nبس بالنسبة ليا ...
15499,1019989115490787200,LY,smsm071990 ALMOGRBE كل 20 تانيه شاب ليبي بيرتا...
15500,1035479791758135168,LY,AboryPro lyranoo85 رانيا عقليتك متخلفة اولا ال...
15501,1035481122921164800,LY,lyranoo85 شكلك متعقدة علشان الراجل لي تحبيه از...


### *Remove Emojis*

In [ ]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [ ]:
filtered_df['tweets'] = filtered_df['tweets'].apply(remove_emoji)
filtered_df.head()

<ipython-input-20-916b9ffa5a77>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['tweets'] = filtered_df['tweets'].apply(remove_emoji)


,id,dialect,tweets
15497,1009754958479151232,LY,tohaAltomy gyyah قليلين ادب ومنافقين لو اختهم ...
15498,1009794751548313600,LY,AlmFaisal الليبيين متقلبين\nبس بالنسبة ليا ان...
15499,1019989115490787200,LY,smsm071990 ALMOGRBE كل 20 تانيه شاب ليبي بيرتا...
15500,1035479791758135168,LY,AboryPro lyranoo85 رانيا عقليتك متخلفة اولا ال...
15501,1035481122921164800,LY,lyranoo85 شكلك متعقدة علشان الراجل لي تحبيه از...


### *Stop Words Elimination*

In [ ]:
nltk.download('stopwords')
stop=stopwords.words('arabic')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
filtered_df['tweets'] = filtered_df['tweets'].apply(lambda x:" ".join(x for x in x.split() if x not in stop))
filtered_df.head()

<ipython-input-22-0f59dbd6df7b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['tweets'] = filtered_df['tweets'].apply(lambda x:" ".join(x for x in x.split() if x not in stop))


,id,dialect,tweets
15497,1009754958479151232,LY,tohaAltomy gyyah قليلين ادب ومنافقين اختهم او ...
15498,1009794751548313600,LY,AlmFaisal الليبيين متقلبين بالنسبة ليا انا ميل...
15499,1019989115490787200,LY,smsm071990 ALMOGRBE 20 تانيه شاب ليبي بيرتاح ل...
15500,1035479791758135168,LY,AboryPro lyranoo85 رانيا عقليتك متخلفة اولا ال...
15501,1035481122921164800,LY,lyranoo85 شكلك متعقدة علشان الراجل تحبيه ازوج ...


### *Remove Diacritics*

In [ ]:
def remove_diacritics(text):
    arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(arabic_diacritics, '', text)
    text = araby.strip_diacritics(text)
    text = araby.strip_shadda(text)
    text = araby.strip_tashkeel(text)
    return text

In [ ]:
filtered_df['tweets'] = filtered_df['tweets'].apply(remove_diacritics)
filtered_df.head()

<ipython-input-24-d6dbb61d3a62>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['tweets'] = filtered_df['tweets'].apply(remove_diacritics)


,id,dialect,tweets
15497,1009754958479151232,LY,tohaAltomy gyyah قليلين ادب ومنافقين اختهم او ...
15498,1009794751548313600,LY,AlmFaisal الليبيين متقلبين بالنسبة ليا انا ميل...
15499,1019989115490787200,LY,smsm071990 ALMOGRBE 20 تانيه شاب ليبي بيرتاح ل...
15500,1035479791758135168,LY,AboryPro lyranoo85 رانيا عقليتك متخلفة اولا ال...
15501,1035481122921164800,LY,lyranoo85 شكلك متعقدة علشان الراجل تحبيه ازوج ...


### *Remove Non-Arabic Characters*

In [ ]:
def remove_non_arabic(text):
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = re.sub('([@A-Za-z0-9_ـــــــــــــ]+)|[^\w\s]|#|http\S+',' ',text)
    text = re.sub(r'\\u[A-Za-z0-9]\\+',' ',text)         #'\\u003c\\1'  error
    return text

In [ ]:
filtered_df['tweets'] = filtered_df['tweets'].apply(remove_non_arabic)
filtered_df.head()

<ipython-input-26-49eb88107966>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['tweets'] = filtered_df['tweets'].apply(remove_non_arabic)


,id,dialect,tweets
15497,1009754958479151232,LY,قليلين ادب ومنافقين اختهم او قريبتهم تتعاك...
15498,1009794751548313600,LY,الليبيين متقلبين بالنسبة ليا انا ميليشياوي ز...
15499,1019989115490787200,LY,تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ...
15500,1035479791758135168,LY,رانيا عقليتك متخلفة اولا الانسان يلي يحتاج...
15501,1035481122921164800,LY,شكلك متعقدة علشان الراجل تحبيه ازوج بنت يتيم...


### *Remove Repeated Characters*

In [ ]:
def remove_repeating_characters(text):
    text = text.strip()
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub(r'(.)\1+', r'\1', text)
    return text

In [ ]:
filtered_df['tweets'] = filtered_df['tweets'].apply(remove_repeating_characters)
filtered_df.head()

<ipython-input-28-6acc143c4d35>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['tweets'] = filtered_df['tweets'].apply(remove_repeating_characters)


,id,dialect,tweets
15497,1009754958479151232,LY,قليلين ادب ومنافقين اختهم او قريبتهم تعاكس تقو...
15498,1009794751548313600,LY,اليبين متقلبين بالنسبه ليا انا ميليشياوي زمان ...
15499,1019989115490787200,LY,تانيه شاب ليبي بيرتاح لبنت مختلفه ويلاحظ انها ...
15500,1035479791758135168,LY,رانيا عقليتك متخلفه اولا الانسان يلي يحتاج اهل...
15501,1035481122921164800,LY,شكلك متعقده علشان الراجل تحبيه ازوج بنت يتيمه ...


### *Lemmatisation*

In [ ]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
filtered_df['tweets'] = filtered_df['tweets'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word, pos=wordnet.VERB) for word in x.split()]))

<ipython-input-30-60bfeca4a2d5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['tweets'] = filtered_df['tweets'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word, pos=wordnet.VERB) for word in x.split()]))


### *Encoding the Target Classes*

In [ ]:
#Encoding output labels
encoding = preprocessing.LabelEncoder()

#using fit transform on the data
y = encoding.fit_transform(filtered_df.dialect.values)

#replace dialec column with the encoded
filtered_df['dialect'] = y

<ipython-input-31-5982a290d3ef>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['dialect'] = y


In [ ]:
encoding
label_map = {label: code for label, code in zip(encoding.classes_, encoding.transform(encoding.classes_))}
label_map

NameError: ignored

### *Saving PreprocessedData*

In [ ]:
#save after preprocessing the tweets
filtered_df.to_csv('preprocessed_tweets.csv')  
     

#copy the csv file to google drive
!cp preprocessed_tweets.csv /content/drive/MyDrive/